#Install required packages

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 24.3 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 6.5 MB 60.4 MB/s 
     |████████████████████████████████| 596 kB 54.2 MB/s 
     |████████████████████████████████| 895 kB 64.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torchinfo

#Load data and prepare for BERT

In [3]:
import os
# from google.colab import drive

import numpy as np
import pandas as pd
import json
import random
import time
import datetime
# from rake_nltk import Rake
import pandas as pd
from scipy import spatial

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from transformers import BertTokenizer, BertModel

import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# df = pd.read_csv('IMDB_Top250Engmovies2_OMDB_Detailed.csv')
df = pd.read_csv('fullset_clean.csv')
df.head()
    
for index, row in df.iterrows():
    df.at[index,'Genre'] = row['Genre'].replace('|',',')

df['Genre_list'] = df['Genre'].map(lambda x: x.split(','))
df['Actors_list'] = df['Actors'].map(lambda x: x.split(',')[:3])
df['Director_List'] = df['Director'].map(lambda x: x.split(','))

for index, row in df.iterrows():
    df.at[index,'Genre_list'] = [x.lower() for x in row['Genre_list']]
    #row['Genre'] = [x.lower().replace(' ','') for x in row['Genre']]
    df.at[index,'Actors_list'] = [x.lower() for x in row['Actors_list']]
    #row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    df.at[index,'Director_List'] = [x.lower() for x in row['Director_List']]
    #row['Director'] = [x.lower().replace(' ','') for x in row['Director']]
    
df.head()
columns = ['Genre_list', 'Director_List', 'Actors_list']

for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'Bag_of_words'] = words
    genre_director = ''
    for col in columns[:2]:
        genre_director += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_genre_director'] = genre_director
    genre_actors = ''
    for col in columns[0:3:2]:
        genre_actors += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_genre_actors'] = genre_actors
    director_actors = ''
    for col in columns[1:]:
        director_actors += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_director_actors'] = director_actors
    
# df = df[['Title','Bag_of_words']]
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors
0,0,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...","[adventure, animation, children, comedy, fantasy]","[tim allen, tom hanks, don rickles]",[john lasseter],adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,john lasseter [SEP] tim allen tom hanks don ...
1,1,Jumanji (1995),"Adventure,Children,Fantasy",Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...","[adventure, children, fantasy]","[jonathan hyde, bradley pierce, robin williams]",[joe johnston],adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] jonathan hyde...,joe johnston [SEP] jonathan hyde bradley pier...
2,2,Grumpier Old Men (1995),"Comedy,Romance",Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...","[comedy, romance]","[jack lemmon, walter matthau, ann-margret ]",[howard deutch],comedy romance [SEP] howard deutch [SEP] jack ...,comedy romance [SEP] howard deutch [SEP],comedy romance [SEP] jack lemmon walter matth...,howard deutch [SEP] jack lemmon walter mattha...
3,3,Waiting to Exhale (1995),"Comedy,Drama,Romance",Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...","[comedy, drama, romance]","[angela bassett, loretta devine, whitney hou...",[forest whitaker],comedy drama romance [SEP] forest whitaker [SE...,comedy drama romance [SEP] forest whitaker [SEP],comedy drama romance [SEP] angela bassett lor...,forest whitaker [SEP] angela bassett loretta ...
4,4,Father of the Bride Part II (1995),Comedy,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",[comedy],"[steve martin, martin short, diane keaton]",[charles shyer],comedy [SEP] charles shyer [SEP] steve martin ...,comedy [SEP] charles shyer [SEP],comedy [SEP] steve martin martin short diane...,charles shyer [SEP] steve martin martin short...


In [5]:
text = list(df['Bag_of_words'])

In [6]:
text

['adventure animation children comedy fantasy [SEP] john lasseter [SEP] tim allen  tom hanks  don rickles [SEP] ',
 'adventure children fantasy [SEP] joe johnston [SEP] jonathan hyde  bradley pierce  robin williams [SEP] ',
 'comedy romance [SEP] howard deutch [SEP] jack lemmon  walter matthau  ann-margret  [SEP] ',
 'comedy drama romance [SEP] forest whitaker [SEP] angela bassett  loretta devine  whitney houston [SEP] ',
 'comedy [SEP] charles shyer [SEP] steve martin  martin short  diane keaton [SEP] ',
 'action crime thriller [SEP] michael mann [SEP] robert de niro  al pacino  val kilmer [SEP] ',
 'comedy romance [SEP] sydney pollack [SEP] harrison ford  greg kinnear  nancy marchand [SEP] ',
 'adventure children [SEP] peter hewitt [SEP] jonathan taylor thomas  brad renfro  eric schweig [SEP] ',
 'action [SEP] peter hyams [SEP] raymond j. barry  powers boothe  jean-claude van damme [SEP] ',
 'action adventure thriller [SEP] martin campbell [SEP] pierce brosnan  sean bean  famke janss

#Setup for BERT model

In [7]:
# Print Torch version
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print('-' * 10)

# Check GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1.10.0+cu111
11.1
8005
----------
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [8]:
seed=17

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.empty_cache()



In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
# Pytorch Dataset - Encodes text with BERT Tokenizer

class Text_Dataset(torch.utils.data.Dataset):
    def __init__(self, text_data, process_length):
        self.text = [tokenizer(text, padding='max_length', 
                              max_length=process_length, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')
                            for text in text_data]

    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        batch_text = self.text[idx]
        
        return batch_text
        

In [11]:
text_length = 16
batch_size = 64

text_dataset = Text_Dataset(text, text_length)

text_loader = torch.utils.data.DataLoader(text_dataset, batch_size=batch_size, shuffle=False)

In [12]:
text_dataset.text[0]

{'input_ids': tensor([[  101,  6172,  7284,  2336,  4038,  5913,   102,  2198, 27333, 15141,
           102,  5199,  5297,  3419,  9180,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
# Define Model
class BERT_embed(nn.Module):

    def __init__(self):
        super(BERT_embed, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertModel.from_pretrained(options_name, output_attentions=False, output_hidden_states=True)

        
    def forward(self, batch_ids, batch_mask):
        output = self.encoder(input_ids=batch_ids, attention_mask=batch_mask)

        return output

In [14]:
testmodel = BERT_embed()
summary(testmodel)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Layer (type:depth-idx)                                  Param #
BERT_embed                                              --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0

In [15]:
# text

# for x in text_loader:
#   print(x)
#   print(x['input_ids'].size())

In [16]:
# Train Function

def train_model(model, dataloaders, num_epochs=1):
    
    model.eval()
    
    est = False
    with torch.no_grad():

      # iterate over dataloader
      for inputs in dataloaders['val']:
          # Send to GPU
          inputs_ids = inputs['input_ids'].squeeze().to(device)
          inputs_mask = inputs['attention_mask'].squeeze().to(device)
          # print(inputs_ids.size())
          # print(labels)
          
          # Fit to model
          outputs = model(inputs_ids, inputs_mask)
          # print(outputs[0].size())

          if est == False:
            # res1=outputs[0]
            res2=outputs[1].detach().cpu()
            # res3=outputs[2]
            est = True
          else:
            # res1 = torch.cat((res1,outputs[0]),0)
            res2 = torch.cat((res2,outputs[1].detach().cpu()),0)
            # res3 = torch.cat((res3,outputs[2]),0)
          # print(res2.size())
              
    return model, outputs, res2

In [17]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':text_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
df['BoW_embedding'] = ''

In [19]:
for index, row in df.iterrows():
    df.at[index,'BoW_embedding'] = res2.detach().numpy()[index]

#Generate BERT embeddings for individual columns

In [20]:
text_length = 16
batch_size = 64

genre_dataset = Text_Dataset(list(df['Genre']), text_length)

genre_loader = torch.utils.data.DataLoader(genre_dataset, batch_size=batch_size, shuffle=False)

In [21]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
df['genre_embedding'] = ''

In [23]:
for index, row in df.iterrows():
    df.at[index,'genre_embedding'] = res2.detach().numpy()[index]

In [24]:
text_length = 16
batch_size = 64

director_dataset = Text_Dataset(list(df['Director']), text_length)

director_loader = torch.utils.data.DataLoader(director_dataset, batch_size=batch_size, shuffle=False)

In [25]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':director_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
df['director_embedding'] = ''

In [27]:
for index, row in df.iterrows():
    df.at[index,'director_embedding'] = res2.detach().numpy()[index]

In [28]:
text_length = 16
batch_size = 64

actors_dataset = Text_Dataset(list(df['Actors']), text_length)

actors_loader = torch.utils.data.DataLoader(actors_dataset, batch_size=batch_size, shuffle=False)

In [29]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
df['actors_embedding'] = ''

In [31]:
for index, row in df.iterrows():
    df.at[index,'actors_embedding'] = res2.detach().numpy()[index]

#Generate BERT embeddings for combination columns

In [32]:
text_length = 16
batch_size = 64

director_actors_dataset = Text_Dataset(list(df['BoW_director_actors']), text_length)

director_actors_loader = torch.utils.data.DataLoader(director_actors_dataset, batch_size=batch_size, shuffle=False)

In [33]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':director_actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
df['director_actors_embedding'] = ''

In [35]:
for index, row in df.iterrows():
    df.at[index,'director_actors_embedding'] = res2.detach().numpy()[index]

In [36]:
text_length = 16
batch_size = 64

genre_actors_dataset = Text_Dataset(list(df['BoW_genre_actors']), text_length)

genre_actors_loader = torch.utils.data.DataLoader(genre_actors_dataset, batch_size=batch_size, shuffle=False)

In [37]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
df['genre_actors_embedding'] = ''

In [39]:
for index, row in df.iterrows():
    df.at[index,'genre_actors_embedding'] = res2.detach().numpy()[index]

In [40]:
text_length = 16
batch_size = 64

genre_director_dataset = Text_Dataset(list(df['BoW_genre_director']), text_length)

genre_director_loader = torch.utils.data.DataLoader(genre_director_dataset, batch_size=batch_size, shuffle=False)

In [41]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_director_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
df['genre_director_embedding'] = ''

In [43]:
for index, row in df.iterrows():
    df.at[index,'genre_director_embedding'] = res2.detach().numpy()[index]

In [44]:
df.head()

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
0,0,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...","[adventure, animation, children, comedy, fantasy]","[tim allen, tom hanks, don rickles]",[john lasseter],adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,john lasseter [SEP] tim allen tom hanks don ...,"[-0.9068614, -0.45405352, -0.80455714, 0.75044...","[-0.8988625, -0.57012534, -0.99096996, 0.91168...","[-0.8687084, -0.30918023, -0.30689827, 0.65527...","[-0.88004625, -0.60375607, -0.98654175, 0.7553...","[-0.8976941, -0.4799076, -0.7705132, 0.7016066...","[-0.9034863, -0.53590953, -0.8715768, 0.737469...","[-0.90445274, -0.47810483, -0.8167057, 0.79308..."
1,1,Jumanji (1995),"Adventure,Children,Fantasy",Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...","[adventure, children, fantasy]","[jonathan hyde, bradley pierce, robin williams]",[joe johnston],adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] jonathan hyde...,joe johnston [SEP] jonathan hyde bradley pier...,"[-0.8920922, -0.3682144, -0.40223628, 0.756093...","[-0.89932764, -0.49810314, -0.98150486, 0.9110...","[-0.7428713, -0.12231341, 0.05766613, 0.381438...","[-0.9077679, -0.58974427, -0.94957626, 0.83750...","[-0.8599445, -0.38748986, -0.42035514, 0.68593...","[-0.916348, -0.40694273, -0.53880054, 0.808105...","[-0.8948911, -0.37579343, -0.5455437, 0.787583..."
2,2,Grumpier Old Men (1995),"Comedy,Romance",Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...","[comedy, romance]","[jack lemmon, walter matthau, ann-margret ]",[howard deutch],comedy romance [SEP] howard deutch [SEP] jack ...,comedy romance [SEP] howard deutch [SEP],comedy romance [SEP] jack lemmon walter matth...,howard deutch [SEP] jack lemmon walter mattha...,"[-0.9042945, -0.46400297, -0.690274, 0.7970322...","[-0.89284724, -0.45970407, -0.9596586, 0.86451...","[-0.82786524, -0.28569412, -0.14196457, 0.6659...","[-0.91705436, -0.5562112, -0.9739642, 0.855578...","[-0.85550654, -0.49105352, -0.8668252, 0.73722...","[-0.92988086, -0.6175473, -0.9531798, 0.882426...","[-0.87577635, -0.38601547, -0.3655597, 0.75878..."
3,3,Waiting to Exhale (1995),"Comedy,Drama,Romance",Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...","[comedy, drama, romance]","[angela bassett, loretta devine, whitney hou...",[forest whitaker],comedy drama romance [SEP] forest whitaker [SE...,comedy drama romance [SEP] forest whitaker [SEP],comedy drama romance [SEP] angela bassett lor...,forest whitaker [SEP] angela bassett loretta ...,"[-0.93603516, -0.5670282, -0.8451405, 0.845204...","[-0.9080508, -0.5593205, -0.98510927, 0.895620...","[-0.9699908, -0.4320408, -0.8304918, 0.9324962...","[-0.90437317, -0.66993964, -0.97878164, 0.8077...","[-0.89529276, -0.5065453, -0.64752454, 0.72272...","[-0.9358029, -0.60015255, -0.87821615, 0.85341...","[-0.8997284, -0.44859746, -0.45308366, 0.81648..."
4,4,Father of the Bride Part II (1995),Comedy,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",[comedy],"[steve martin, martin short, diane keaton]",[charles shyer],comedy [SEP] charles shyer [SEP] steve martin ...,comedy [SEP] charles shyer [SEP],comedy [SEP] steve martin martin short diane...,charles shyer [SEP] steve martin martin short...,"[-0.9054226, -0.5058742, -0.7242107, 0.7743211...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.87651086, -0.34650725, -0.5925919, 0.68864...","[-0.8820019, -0.60759795, -0.97453606, 0.80612...","[-0.89944583, -0.52649724, -0.7816844, 0.7549,

#Concatenate BERT embedding for individual columns to create combinations

In [54]:
df['genre_director_concat_embedding'] = ''

for index, row in df.iterrows():
    df.at[index,'genre_director_concat_embedding'] = np.concatenate((df.genre_embedding[index],df.director_embedding[index]), axis=0)

In [60]:
df['genre_actors_concat_embedding'] = ''

for index, row in df.iterrows():
    df.at[index,'genre_actors_concat_embedding'] = np.concatenate((df.genre_embedding[index],df.actors_embedding[index]), axis=0)

In [61]:
df['director_actors_concat_embedding'] = ''

for index, row in df.iterrows():
    df.at[index,'director_actors_concat_embedding'] = np.concatenate((df.director_embedding[index],df.actors_embedding[index]), axis=0)

In [62]:
df['full_concat_embedding'] = ''

for index, row in df.iterrows():
    df.at[index,'full_concat_embedding'] = np.concatenate((df.genre_embedding[index],df.director_embedding[index],df.actors_embedding[index]), axis=0)

In [63]:
df.head()

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,...,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding,genre_director_concat_embedding,genre_actors_concat_embedding,director_actors_concat_embedding,full_concat_embedding
0,0,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...","[adventure, animation, children, comedy, fantasy]","[tim allen, tom hanks, don rickles]",[john lasseter],adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,...,"[-0.8988625, -0.57012534, -0.99096996, 0.91168...","[-0.8687084, -0.30918023, -0.30689827, 0.65527...","[-0.88004625, -0.60375607, -0.98654175, 0.7553...","[-0.8976941, -0.4799076, -0.7705132, 0.7016066...","[-0.9034863, -0.53590953, -0.8715768, 0.737469...","[-0.90445274, -0.47810483, -0.8167057, 0.79308...","[-0.8988625, -0.57012534, -0.99096996, 0.91168...","[-0.8988625, -0.57012534, -0.99096996, 0.91168...","[-0.8687084, -0.30918023, -0.30689827, 0.65527...","[-0.8988625, -0.57012534, -0.99096996, 0.91168..."
1,1,Jumanji (1995),"Adventure,Children,Fantasy",Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...","[adventure, children, fantasy]","[jonathan hyde, bradley pierce, robin williams]",[joe johnston],adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] joe johnston ...,...,"[-0.89932764, -0.49810314, -0.98150486, 0.9110...","[-0.7428713, -0.12231341, 0.05766613, 0.381438...","[-0.9077679, -0.58974427, -0.94957626, 0.83750...","[-0.8599445, -0.38748986, -0.42035514, 0.68593...","[-0.916348, -0.40694273, -0.53880054, 0.808105...","[-0.8948911, -0.37579343, -0.5455437, 0.787583...","[-0.89932764, -0.49810314, -0.98150486, 0.9110...","[-0.89932764, -0.49810314, -0.98150486, 0.9110...","[-0.7428713, -0.12231341, 0.05766613, 0.381438...","[-0.89932764, -0.49810314, -0.98150486, 0.9110..."
2,2,Grumpier Old Men (1995),"Comedy,Romance",Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...","[comedy, romance]","[jack lemmon, walter matthau, ann-margret ]",[howard deutch],comedy romance [SEP] howard deutch [SEP] jack ...,comedy romance [SEP] howard deutch [SEP],...,"[-0.89284724, -0.45970407, -0.9596586, 0.86451...","[-0.82786524, -0.28569412, -0.14196457, 0.6659...","[-0.91705436, -0.5562112, -0.9739642, 0.855578...","[-0.85550654, -0.49105352, -0.8668252, 0.73722...","[-0.92988086, -0.6175473, -0.9531798, 0.882426...","[-0.87577635, -0.38601547, -0.3655597, 0.75878...","[-0.89284724, -0.45970407, -0.9596586, 0.86451...","[-0.89284724, -0.45970407, -0.9596586, 0.86451...","[-0.82786524, -0.28569412, -0.14196457, 0.6659...","[-0.89284724, -0.45970407, -0.9596586, 0.86451..."
3,3,Waiting to Exhale (1995),"Comedy,Drama,Romance",Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...","[comedy, drama, romance]","[angela bassett, loretta devine, whitney hou...",[forest whitaker],comedy drama romance [SEP] forest whitaker [SE...,comedy drama romance [SEP] forest whitaker [SEP],...,"[-0.9080508, -0.5593205, -0.98510927, 0.895620...","[-0.9699908, -0.4320408, -0.8304918, 0.9324962...","[-0.90437317, -0.66993964, -0.97878164, 0.8077...","[-0.89529276, -0.5065453, -0.64752454, 0.72272...","[-0.9358029, -0.60015255, -0.87821615, 0.85341...","[-0.8997284, -0.44859746, -0.45308366, 0.81648...","[-0.9080508, -0.5593205, -0.98510927, 0.895620...","[-0.9080508, -0.5593205, -0.98510927, 0.895620...","[-0.9699908, -0.4320408, -0.8304918, 0.9324962...","[-0.9080508, -0.5593205, -0.98510927, 0.895620..."
4,4,Father of the Bride Part II (1995),Comedy,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",[comedy],"[steve martin, martin short, diane keaton]",[charles shyer],comedy [SEP] charles shyer [SEP] steve martin ...,comedy [SEP] charles shyer [SEP],...,"[-0.9387212, -0.56645083, -0.8

#Test Cosine similarity for BERT embeddings

In [ ]:
# cosine_sim = cosine_similarity(list(df['embedding']), list(df['embedding']))
# #print(cosine_sim)

# indices = pd.Series(df['Title'])

# def recommend(title, cosine_sim = cosine_sim):
#     recommended_movies = []
#     idx = indices[indices == title].index[0]
#     score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
#     top_10_indices = list(score_series.iloc[1:11].index)
    
#     for i in top_10_indices:
#         recommended_movies.append(list(df['Title'])[i])
    
#     return recommended_movies

# print(recommend('The Godfather'))

In [ ]:
search_phrase="crime drama [SEP] francis ford coppola [SEP] marlon brando"

search_tok=tokenizer(search_phrase, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out = model(search_tok['input_ids'].to(device), search_tok['attention_mask'].to(device))

In [ ]:
cosine_sim_search = cosine_similarity(np.vstack(df.BoW_embedding), search_out[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]","[marlon brando, al pacino, james caan]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] m...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] marlon brando al pacino ja...,francis ford coppola [SEP] marlon brando al p...,"[-0.9124511, -0.4561468, -0.668873, 0.79738057...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.88796747, -0.5574769, -0.9324712, 0.800263...","[-0.9056314, -0.41298485, -0.6861082, 0.782320...","[-0.93583083, -0.52514285, -0.7718601, 0.82903...","[-0.91062135, -0.44303492, -0.53099483, 0.8139..."
743,743,Jack (1996),"Comedy,Drama",Francis Ford Coppola,"Brian Kerwin, Robin Williams, Diane Lane, Jenn...","[comedy, drama]","[brian kerwin, robin williams, diane lane]",[francis ford coppola],comedy drama [SEP] francis ford coppola [SEP] ...,comedy drama [SEP] francis ford coppola [SEP],comedy drama [SEP] brian kerwin robin william...,francis ford coppola [SEP] brian kerwin robin...,"[-0.9300826, -0.5037794, -0.6723434, 0.829361,...","[-0.90341103, -0.48435685, -0.90629137, 0.8427...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.94045836, -0.70812804, -0.9808046, 0.86297...","[-0.87570965, -0.44571155, -0.4774299, 0.70948...","[-0.91895914, -0.5220835, -0.77870417, 0.81315...","[-0.9244548, -0.47262412, -0.64834684, 0.83451..."
36210,36210,The Count of Monte-Cristo (1974),"Action,Adventure,Drama",David Greene,"Richard Chamberlain,Kate Nelligan,Isabelle De ...","[action, adventure, drama]","[richard chamberlain, kate nelligan, isabelle ...",[david greene],action adventure drama [SEP] david greene [SEP...,action adventure drama [SEP] david greene [SEP],action adventure drama [SEP] richard chamberla...,david greene [SEP] richard chamberlain kate ne...,"[-0.91207784, -0.48735285, -0.65120155, 0.7954...","[-0.93454224, -0.5602194, -0.973209, 0.9251991...","[-0.46349952, -0.054932456, 0.83025455, 0.0992...","[-0.89394027, -0.5977378, -0.9544844, 0.769176...","[-0.8643197, -0.43860275, -0.5001003, 0.694376...","[-0.92873317, -0.5275629, -0.78993785, 0.83647...","[-0.9359472, -0.530855, -0.7529209, 0.8607885,..."
23796,23796,China Girl (1987),"Crime,Drama,Romance",Abel Ferrara,"James Russo,Richard Panebianco,David Caruso,Sa...","[crime, drama, romance]","[james russo, richard panebianco, david caruso]",[abel ferrara],crime drama romance [SEP] abel ferrara [SEP] j...,crime drama romance [SEP] abel ferrara [SEP],crime drama romance [SEP] james russo richard ...,abel ferrara [SEP] james russo richard panebia...,"[-0.9088795, -0.48056227, -0.6890371, 0.803001...","[-0.8951508, -0.6372436, -0.99356914, 0.913856...","[-0.86713946, -0.3153721, 0.26026073, 0.633872...","[-0.9005756, -0.5510442, -0.9846165, 0.8814346...","[-0.89190835, -0.4522677, -0.4495127, 0.736924...","[-0.92507184, -0.5229723, -0.84791595, 0.85288...","[-0.9214453, -0.48701397, -0.602351, 0.8322243..."
783,783,American Buffalo (1996),"Crime,Drama",Michael Corrente,"Dennis Franz, Dustin Hoffman, Sean Nelson","[crime, drama]","[dennis franz, dustin hoffman, sean nelson]",[michael corrente],crime drama [SEP] michael corrente [SEP] denni...,crime drama [SEP] michael corrente [SEP],crime drama [SEP] dennis franz dustin hoffman...,michael corrente [SEP] dennis franz dustin ho...,"[-0.91709834, -0.49358177, -0.74666226, 0.7940...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8287955, -0.29479554, -0.24427372, 0.57922...","[-0.9375564, -0.6801224, -0.97719413, 0.840070...","[-0.9086757, -0.45047095, -0.73195624, 0.74563...","[-0.924

#Test BERT embeddings for Individual columns

In [ ]:
search_phrase_genre="crime drama"

search_tok_genre=tokenizer(search_phrase_genre, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre = model(search_tok_genre['input_ids'].to(device), search_tok_genre['attention_mask'].to(device))

In [ ]:
cosine_sim_search_genre = cosine_similarity(np.vstack(df.genre_embedding), search_out_genre[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
25662,25662,Deliverance Creek (2014),Drama,Jon Amiel,"Lauren Ambrose,Wes Ramsey,Christopher Backus,R...",[drama],"[lauren ambrose, wes ramsey, christopher backus]",[jon amiel],drama [SEP] jon amiel [SEP] lauren ambrose wes...,drama [SEP] jon amiel [SEP],drama [SEP] lauren ambrose wes ramsey christop...,jon amiel [SEP] lauren ambrose wes ramsey chri...,"[-0.89953065, -0.45317027, -0.39470017, 0.7432...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.8054115, -0.37052855, -0.2544165, 0.593378...","[-0.82209885, -0.60680497, -0.9927784, 0.84705...","[-0.8725585, -0.4442692, -0.4994698, 0.7026132...","[-0.92751133, -0.5183296, -0.61729836, 0.80681...","[-0.8948341, -0.39424774, -0.26691175, 0.76968..."
38961,38961,Vampire in Love (2015),Drama,Mai Suzuki,"Mirei Kiritani,Shota Totsuka,Seiichi Tanabe,Ne...",[drama],"[mirei kiritani, shota totsuka, seiichi tanabe]",[mai suzuki],drama [SEP] mai suzuki [SEP] mirei kiritani sh...,drama [SEP] mai suzuki [SEP],drama [SEP] mirei kiritani shota totsuka seiic...,mai suzuki [SEP] mirei kiritani shota totsuka ...,"[-0.9082873, -0.47731364, -0.71952355, 0.81259...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.8254441, -0.19854681, 0.16875838, 0.582405...","[-0.90554523, -0.58859515, -0.9919392, 0.87467...","[-0.86339384, -0.44243148, -0.8077887, 0.78130...","[-0.9203686, -0.53793937, -0.8083023, 0.843551...","[-0.897138, -0.37075773, -0.26017776, 0.768132..."
46294,46294,Kochavva Paulo Ayyappa Coelho (2016),Drama,Sidhartha Siva,"Kunchacko Boban,Rudraksh Sudheesh,Nedumudi Ven...",[drama],"[kunchacko boban, rudraksh sudheesh, nedumudi ...",[sidhartha siva],drama [SEP] sidhartha siva [SEP] kunchacko bob...,drama [SEP] sidhartha siva [SEP],drama [SEP] kunchacko boban rudraksh sudheesh ...,sidhartha siva [SEP] kunchacko boban rudraksh ...,"[-0.8844829, -0.4814467, -0.69602096, 0.754643...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.86602795, -0.429622, -0.35515386, 0.693870...","[-0.91793954, -0.5303115, -0.9936513, 0.864716...","[-0.812184, -0.43741223, -0.64612085, 0.528058...","[-0.9069429, -0.514922, -0.8244991, 0.7865244,...","[-0.91633934, -0.50909, -0.60572124, 0.8213804..."
38946,38946,Alone in Berlin (2016),Drama,Vincent Pérez,"Emma Thompson,Daniel Brühl,Brendan Gleeson,Mik...",[drama],"[emma thompson, daniel brühl, brendan gleeson]",[vincent pérez],drama [SEP] vincent pérez [SEP] emma thompson ...,drama [SEP] vincent pérez [SEP],drama [SEP] emma thompson daniel brühl brendan...,vincent pérez [SEP] emma thompson daniel brühl...,"[-0.92474264, -0.5098804, -0.70508, 0.81712645...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.80478156, -0.2321129, 0.2027001, 0.5668568...","[-0.88082105, -0.51569116, -0.9759039, 0.84752...","[-0.90593344, -0.44266784, -0.6749019, 0.78077...","[-0.9305928, -0.53054285, -0.74059343, 0.83287...","[-0.91572046, -0.4409757, -0.49078757, 0.83181..."
26287,26287,The Moromete Family (1987),Drama,Stere Gulea,"Victor Rebengiuc,Luminita Gheorghiu,Gina Patri...",[drama],"[victor rebengiuc, luminita gheorghiu, gina pa...",[stere gulea],drama [SEP] stere gulea [SEP] victor rebengiuc...,drama [SEP] stere gulea [SEP],drama [SEP] victor rebengiuc luminita gheorghi...,stere gulea [SEP] victor rebengiuc luminita gh...,"[-0.926957, -0.42347682, -0.5960797, 0.8354261...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.7762668, -0.29311427, 0.029514372, 0.69250...","[-0.8613139, -0.34675825, -0.9523423, 0.830575...","[-0.9031745, -0.33604544, -0.7448367, 0.840475...","[-0.9100698, -0.38574743, -0.7268888, 0.825943...","[-0.920717, -0.38988554, -0.48037097, 0.858413..."
9155,9155,"Milwaukee, Minnesota (2003)",Drama,Allan Mindel,"Troy Garity, Randy 

In [ ]:
search_phrase_director="francis ford coppola"

search_tok_director=tokenizer(search_phrase_director, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_director = model(search_tok_director['input_ids'].to(device), search_tok_director['attention_mask'].to(device))

In [ ]:
cosine_sim_search_director = cosine_similarity(np.vstack(df.director_embedding), search_out_director[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_director.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
743,743,Jack (1996),"Comedy,Drama",Francis Ford Coppola,"Brian Kerwin, Robin Williams, Diane Lane, Jenn...","[comedy, drama]","[brian kerwin, robin williams, diane lane]",[francis ford coppola],comedy drama [SEP] francis ford coppola [SEP] ...,comedy drama [SEP] francis ford coppola [SEP],comedy drama [SEP] brian kerwin robin william...,francis ford coppola [SEP] brian kerwin robin...,"[-0.9300826, -0.5037794, -0.6723434, 0.829361,...","[-0.90341103, -0.48435685, -0.90629137, 0.8427...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.94045836, -0.70812804, -0.9808046, 0.86297...","[-0.87570965, -0.44571155, -0.4774299, 0.70948...","[-0.91895914, -0.5220835, -0.77870417, 0.81315...","[-0.9244548, -0.47262412, -0.64834684, 0.83451..."
5952,5952,One from the Heart (1982),"Drama,Romance",Francis Ford Coppola,"Frederic Forrest, Teri Garr, Raul Julia, Nasta...","[drama, romance]","[frederic forrest, teri garr, raul julia]",[francis ford coppola],drama romance [SEP] francis ford coppola [SEP]...,drama romance [SEP] francis ford coppola [SEP],drama romance [SEP] frederic forrest teri gar...,francis ford coppola [SEP] frederic forrest t...,"[-0.8807221, -0.3880829, -0.4990447, 0.7896178...","[-0.89984787, -0.44744048, -0.96033573, 0.8897...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.8210736, -0.38432306, -0.9298712, 0.779208...","[-0.8708472, -0.36284125, -0.43557903, 0.67098...","[-0.8799921, -0.4238392, -0.62310743, 0.769745...","[-0.89329785, -0.40694097, -0.45446214, 0.7957..."
3608,3608,"Conversation, The (1974)","Drama,Mystery",Francis Ford Coppola,"Gene Hackman, John Cazale, Frederic Forrest, A...","[drama, mystery]","[gene hackman, john cazale, frederic forrest]",[francis ford coppola],drama mystery [SEP] francis ford coppola [SEP]...,drama mystery [SEP] francis ford coppola [SEP],drama mystery [SEP] gene hackman john cazale ...,francis ford coppola [SEP] gene hackman john ...,"[-0.87459004, -0.3895415, -0.44265065, 0.74895...","[-0.85457647, -0.39523104, -0.86477685, 0.8199...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9293822, -0.6691327, -0.99408424, 0.861445...","[-0.92137945, -0.48980555, -0.8624548, 0.80641...","[-0.8883384, -0.43648264, -0.66936237, 0.74162...","[-0.8622598, -0.3556443, -0.19715042, 0.736888..."
4425,4425,Tucker: The Man and His Dream (1988),Drama,Francis Ford Coppola,"Jeff Bridges, Frederic Forrest, Martin Landau,...",[drama],"[jeff bridges, frederic forrest, martin landau]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] jeff br...,drama [SEP] francis ford coppola [SEP],drama [SEP] jeff bridges frederic forrest ma...,francis ford coppola [SEP] jeff bridges frede...,"[-0.9051331, -0.41575477, -0.4098316, 0.768512...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.91164315, -0.59014595, -0.94301134, 0.7848...","[-0.8757097, -0.3812641, -0.35433483, 0.724260...","[-0.9148313, -0.41901135, -0.46939766, 0.79699...","[-0.907878, -0.42283192, -0.44207257, 0.798485..."
14739,14739,"Rain People, The (1969)",Drama,Francis Ford Coppola,"James Caan, Shirley Knight, Robert Duvall",[drama],"[james caan, shirley knight, robert duvall]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] james c...,drama [SEP] francis ford coppola [SEP],drama [SEP] james caan shirley knight robert...,francis ford coppola [SEP] james caan shirley...,"[-0.8741493, -0.3990764, -0.3307979, 0.7316348...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.94587183, -0.6602034, -0.9625455, 0.858021...","[-0.9011832, -0.4551079, -0.45746538, 0.699184...","[-0.88366073, -0.42016616, 

In [ ]:
search_phrase_actors="marlon brando"

search_tok_actors=tokenizer(search_phrase_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_actors = model(search_tok_actors['input_ids'].to(device), search_tok_actors['attention_mask'].to(device))

In [ ]:
cosine_sim_search_actors = cosine_similarity(np.vstack(df.actors_embedding), search_out_actors[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_actors.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
9228,9228,Bukowski: Born into This (2003),Documentary,John Dullaghan,Charles Bukowski,[documentary],[charles bukowski],[john dullaghan],documentary [SEP] john dullaghan [SEP] charles...,documentary [SEP] john dullaghan [SEP],documentary [SEP] charles bukowski [SEP],john dullaghan [SEP] charles bukowski [SEP],"[-0.8632827, -0.34444803, 0.12437683, 0.691908...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.7869288, -0.21070111, -0.106383756, 0.5896...","[-0.91328317, -0.47187495, -0.6175246, 0.81673...","[-0.8104001, -0.28605205, -0.14997505, 0.66880...","[-0.8878547, -0.37836432, -0.024823502, 0.7340...","[-0.8318931, -0.29728636, 0.26635158, 0.638975..."
18646,18646,Carol Channing: Larger Than Life (2012),"Comedy,Documentary,Musical",Dori Berinstein,Carol Channing,"[comedy, documentary, musical]",[carol channing],[dori berinstein],comedy documentary musical [SEP] dori berinste...,comedy documentary musical [SEP] dori berinste...,comedy documentary musical [SEP] carol channin...,dori berinstein [SEP] carol channing [SEP],"[-0.9041953, -0.4001539, -0.36883768, 0.752148...","[-0.9434678, -0.6072608, -0.9725319, 0.8943805...","[-0.8000567, -0.22490527, 0.4960199, 0.6259435...","[-0.936964, -0.4185827, -0.77783525, 0.8290345...","[-0.8248428, -0.31937233, 0.12242878, 0.659324...","[-0.90985566, -0.4051483, -0.41993767, 0.73680...","[-0.91915995, -0.46411088, -0.61213124, 0.8003..."
36753,36753,Star Games (1998),"Adventure,Children,Sci-Fi",Greydon Clark,Tony Curtis,"[adventure, children, sci-fi]",[tony curtis],[greydon clark],adventure children sci-fi [SEP] greydon clark ...,adventure children sci-fi [SEP] greydon clark ...,adventure children sci-fi [SEP] tony curtis [S...,greydon clark [SEP] tony curtis [SEP],"[-0.8851575, -0.48308071, -0.75328034, 0.74889...","[-0.7778927, -0.47610217, -0.96737325, 0.78399...","[-0.835925, -0.2283621, 0.11615332, 0.6829073,...","[-0.9038266, -0.51789176, -0.5996438, 0.595371...","[-0.85224026, -0.35506892, -0.18617658, 0.6648...","[-0.89536434, -0.5838916, -0.8695237, 0.779074...","[-0.9080944, -0.47724393, -0.8559084, 0.830322..."
48556,48556,Beuys (2017),Documentary,Andres Veiel,Joseph Beuys,[documentary],[joseph beuys],[andres veiel],documentary [SEP] andres veiel [SEP] joseph be...,documentary [SEP] andres veiel [SEP],documentary [SEP] joseph beuys [SEP],andres veiel [SEP] joseph beuys [SEP],"[-0.91273993, -0.3759817, -0.22719994, 0.76564...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.94660676, -0.3652507, -0.5085593, 0.800422...","[-0.9401305, -0.47787195, -0.7407714, 0.857372...","[-0.9377446, -0.40711543, -0.56120634, 0.81591...","[-0.9307168, -0.44006142, -0.47599992, 0.81874...","[-0.9130303, -0.35244334, -0.16440374, 0.77438..."
39890,39890,Joe Rogan: Rocky Mountain High (2014),Comedy,Ryan Polito,Joe Rogan,[comedy],[joe rogan],[ryan polito],comedy [SEP] ryan polito [SEP] joe rogan [SEP],comedy [SEP] ryan polito [SEP],comedy [SEP] joe rogan [SEP],ryan polito [SEP] joe rogan [SEP],"[-0.89193785, -0.41751117, -0.24893883, 0.7454...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.8943292, -0.30241567, -0.2815005, 0.712685...","[-0.8923787, -0.44980913, -0.5916888, 0.725004...","[-0.8514758, -0.3213344, -0.16100605, 0.634348...","[-0.90986294, -0.46926695, -0.3993604, 0.76680...","[-0.88758194, -0.39920378, -0.18934739, 0.7643..."
39320,39320,Joe Rogan: Live (2006),Comedy,Michael Blieden,Joe Rogan,[comedy],[joe rogan],[michael blieden],comedy [SEP] michael blieden [SEP] joe rogan [...,comedy [SEP] michael blieden [SEP],comedy [SEP] joe rogan [SEP],michael blieden [SEP] joe rogan [SEP],"[-0.8734685, -0.42108017, -0.31624728, 0.69569...","[-0.9387212, -0.56645083, -0.8519647, 0.8

#Test combination BERT embeddings

In [ ]:
search_phrase_genre_director="crime drama [SEP] francis ford coppola"

search_tok_genre_director=tokenizer(search_phrase_genre_director, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre_director = model(search_tok_genre_director['input_ids'].to(device), search_tok_genre_director['attention_mask'].to(device))

In [ ]:
cosine_sim_search_genre_director = cosine_similarity(np.vstack(df.genre_director_embedding), search_out_genre_director[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre_director.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
5796,5796,Gangs of New York (2002),"Crime,Drama",Martin Scorsese,"Leonardo DiCaprio, Daniel Day-Lewis, Cameron D...","[crime, drama]","[leonardo dicaprio, daniel day-lewis, camero...",[martin scorsese],crime drama [SEP] martin scorsese [SEP] leonar...,crime drama [SEP] martin scorsese [SEP],crime drama [SEP] leonardo dicaprio daniel da...,martin scorsese [SEP] leonardo dicaprio danie...,"[-0.95107085, -0.520219, -0.76897407, 0.873902...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8758811, -0.37692434, -0.022983076, 0.6479...","[-0.94020885, -0.5865772, -0.98064375, 0.88464...","[-0.9492274, -0.5855761, -0.8659955, 0.8948852...","[-0.9399922, -0.54787576, -0.8477248, 0.863471...","[-0.9540982, -0.5407269, -0.711162, 0.88752866..."
7549,7549,Mean Streets (1973),"Crime,Drama",Martin Scorsese,"Robert De Niro, Harvey Keitel, David Carradine...","[crime, drama]","[robert de niro, harvey keitel, david carrad...",[martin scorsese],crime drama [SEP] martin scorsese [SEP] robert...,crime drama [SEP] martin scorsese [SEP],crime drama [SEP] robert de niro harvey keite...,martin scorsese [SEP] robert de niro harvey k...,"[-0.918061, -0.4667398, -0.5064786, 0.82602906...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8758811, -0.37692434, -0.022983076, 0.6479...","[-0.91790754, -0.6114952, -0.9845845, 0.890974...","[-0.9341164, -0.53019476, -0.64579403, 0.81474...","[-0.9388962, -0.5484542, -0.7291734, 0.8417468...","[-0.9540982, -0.5407269, -0.711162, 0.88752866..."
15,15,Casino (1995),"Crime,Drama",Martin Scorsese,"Robert De Niro, Joe Pesci, James Woods, Sharon...","[crime, drama]","[robert de niro, joe pesci, james woods]",[martin scorsese],crime drama [SEP] martin scorsese [SEP] robert...,crime drama [SEP] martin scorsese [SEP],crime drama [SEP] robert de niro joe pesci j...,martin scorsese [SEP] robert de niro joe pesc...,"[-0.91622865, -0.4584852, -0.5189044, 0.818048...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8758811, -0.37692434, -0.022983076, 0.6479...","[-0.90458345, -0.5954771, -0.97678864, 0.87324...","[-0.92955315, -0.5531311, -0.772004, 0.8537627...","[-0.9129321, -0.51151574, -0.6641731, 0.808491...","[-0.9540982, -0.5407269, -0.711162, 0.88752866..."
1170,1170,Goodfellas (1990),"Crime,Drama",Martin Scorsese,"Robert De Niro, Ray Liotta, Joe Pesci, Lorrain...","[crime, drama]","[robert de niro, ray liotta, joe pesci]",[martin scorsese],crime drama [SEP] martin scorsese [SEP] robert...,crime drama [SEP] martin scorsese [SEP],crime drama [SEP] robert de niro ray liotta ...,martin scorsese [SEP] robert de niro ray liot...,"[-0.93763304, -0.5116118, -0.6763859, 0.855911...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8758811, -0.37692434, -0.022983076, 0.6479...","[-0.9329656, -0.6667259, -0.9615686, 0.866897,...","[-0.9255097, -0.5841254, -0.8540772, 0.8343237...","[-0.9220011, -0.5729488, -0.8168441, 0.8183913...","[-0.9540982, -0.5407269, -0.711162, 0.88752866..."
53344,53344,The Irishman (2019),"Crime,Drama",Martin Scorsese,"Robert De Niro,Al Pacino,Joe Pesci,Harvey Keit...","[crime, drama]","[robert de niro, al pacino, joe pesci]",[martin scorsese],crime drama [SEP] martin scorsese [SEP] robert...,crime drama [SEP] martin scorsese [SEP],crime drama [SEP] robert de niro al pacino joe...,martin scorsese [SEP] robert de niro al pacino...,"[-0.92428935, -0.49667668, -0.617239, 0.821081...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.8758811, -0.37692434, -0.022983076, 0.6479...","[-0.89705944, -0.6545123, -0.99458593, 0.89096...","[-0.8522463, -0.404884, -0.45893484, 0.7003238...","[-0.93359846, -0.6089617, -0.8656832, 0.853690...","[-0.9540982, -0.5407269, -0.711162, 0.88752866...

In [ ]:
search_phrase_genre_actors="crime drama [SEP] marlon brando"

search_tok_genre_actors=tokenizer(search_phrase_genre_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre_actors = model(search_tok_genre_actors['input_ids'].to(device), search_tok_genre_actors['attention_mask'].to(device))

In [ ]:
cosine_sim_search_genre_actors = cosine_similarity(np.vstack(df.genre_actors_embedding), search_out_genre_actors[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre_actors.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]","[marlon brando, al pacino, james caan]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] m...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] marlon brando al pacino ja...,francis ford coppola [SEP] marlon brando al p...,"[-0.9124511, -0.4561468, -0.668873, 0.79738057...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.88796747, -0.5574769, -0.9324712, 0.800263...","[-0.9056314, -0.41298485, -0.6861082, 0.782320...","[-0.93583083, -0.52514285, -0.7718601, 0.82903...","[-0.91062135, -0.44303492, -0.53099483, 0.8139..."
32952,32952,Steelyard Blues (1973),"Comedy,Crime,Drama",Alan Myerson,"Donald Sutherland,Peter Boyle,Jane Fonda,Howar...","[comedy, crime, drama]","[donald sutherland, peter boyle, jane fonda]",[alan myerson],comedy crime drama [SEP] alan myerson [SEP] do...,comedy crime drama [SEP] alan myerson [SEP],comedy crime drama [SEP] donald sutherland pet...,alan myerson [SEP] donald sutherland peter boy...,"[-0.915298, -0.47375983, -0.6912867, 0.8236065...","[-0.9050065, -0.5585141, -0.960561, 0.88835573...","[-0.88124037, -0.36876255, -0.5124708, 0.73855...","[-0.9018101, -0.5109636, -0.9109065, 0.7713609...","[-0.9009846, -0.44306022, -0.60687995, 0.74020...","[-0.93682283, -0.53158695, -0.76151633, 0.8352...","[-0.91530406, -0.51204586, -0.71713233, 0.8226..."
423,423,Carlito's Way (1993),"Crime,Drama",Brian De Palma,"Al Pacino, Sean Penn, Penelope Ann Miller, Joh...","[crime, drama]","[al pacino, sean penn, penelope ann miller]",[brian de palma],crime drama [SEP] brian de palma [SEP] al paci...,crime drama [SEP] brian de palma [SEP],crime drama [SEP] al pacino sean penn penelo...,brian de palma [SEP] al pacino sean penn pen...,"[-0.90994406, -0.48673216, -0.6355186, 0.78374...","[-0.87593293, -0.50445807, -0.8962387, 0.80006...","[-0.84804255, -0.26358995, -0.4038505, 0.56795...","[-0.9191719, -0.65236396, -0.98345274, 0.86228...","[-0.86860484, -0.4103516, -0.26214957, 0.63795...","[-0.9330533, -0.5481016, -0.7542494, 0.8291294...","[-0.9336319, -0.5001749, -0.7333737, 0.8351466..."
4334,4334,Above the Law (1988),"Action,Crime,Drama",Andrew Davis,"Ron Dean, Steven Seagal, Henry Silva, Pam Grier","[action, crime, drama]","[ron dean, steven seagal, henry silva]",[andrew davis],action crime drama [SEP] andrew davis [SEP] ro...,action crime drama [SEP] andrew davis [SEP],action crime drama [SEP] ron dean steven seag...,andrew davis [SEP] ron dean steven seagal he...,"[-0.9269688, -0.49708682, -0.70264626, 0.81329...","[-0.91923726, -0.5632013, -0.9638532, 0.901466...","[-0.7818563, -0.22395977, 0.20033927, 0.576995...","[-0.86241007, -0.58292973, -0.98115385, 0.7851...","[-0.85442555, -0.3559466, -0.43323994, 0.65045...","[-0.9408565, -0.5343398, -0.7682996, 0.8401463...","[-0.93605006, -0.50568914, -0.6369958, 0.86449..."
8660,8660,"Spook Who Sat by the Door, The (1973)","Action,Crime,Drama",Ivan Dixon,"Lawrence Cook, Paula Kelly","[action, crime, drama]","[lawrence cook, paula kelly]",[ivan dixon],action crime drama [SEP] ivan dixon [SEP] lawr...,action crime drama [SEP] ivan dixon [SEP],action crime drama [SEP] lawrence cook paula ...,ivan dixon [SEP] lawrence cook paula kelly [S...,"[-0.9212233, -0.49290428, -0.7027701, 0.824402...","[-0.91923726, -0.5632013, -0.9638532, 0.901466...","[-0.7435626, -0.17243074, -0.11435668, 0.54981...","[-0.8486881, -0.5322655, -0.9555047, 0.7545674...","[-0.86799055, -0.3889626, -0.55505764, 0.70009...","[-0.9384705, -0.54029584, -0.7587655, 0.862444...","[-0.93630993, -0.48

In [ ]:
search_phrase_director_actors="francis ford coppola [SEP] marlon brando"

search_tok_director_actors=tokenizer(search_phrase_director_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_director_actors = model(search_tok_director_actors['input_ids'].to(device), search_tok_director_actors['attention_mask'].to(device))

In [ ]:
cosine_sim_search_director_actors = cosine_similarity(np.vstack(df.director_actors_embedding), search_out_director_actors[1].detach().cpu().numpy())

In [ ]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim = cosine_sim_search):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_director_actors.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
1916,1916,"Godfather: Part III, The (1990)","Crime,Drama,Mystery,Thriller",Francis Ford Coppola,"Al Pacino, Diane Keaton, Talia Shire, Andy Gar...","[crime, drama, mystery, thriller]","[al pacino, diane keaton, talia shire]",[francis ford coppola],crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller [SEP] al pacino ...,francis ford coppola [SEP] al pacino diane ke...,"[-0.8999591, -0.44364092, -0.61394703, 0.77145...","[-0.88857543, -0.687736, -0.9892572, 0.8864184...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.91834736, -0.6246187, -0.9715122, 0.786767...","[-0.912915, -0.4990247, -0.7544489, 0.7603222,...","[-0.92526126, -0.5355736, -0.7029956, 0.811331...","[-0.9047692, -0.43409953, -0.46472472, 0.79389..."
1165,1165,Apocalypse Now (1979),"Action,Drama,War",Francis Ford Coppola,"Marlon Brando, Robert Duvall, Frederic Forrest...","[action, drama, war]","[marlon brando, robert duvall, frederic forr...",[francis ford coppola],action drama war [SEP] francis ford coppola [S...,action drama war [SEP] francis ford coppola [S...,action drama war [SEP] marlon brando robert d...,francis ford coppola [SEP] marlon brando robe...,"[-0.9069395, -0.48042327, -0.58944505, 0.80357...","[-0.9148726, -0.61122835, -0.9817057, 0.931527...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9322366, -0.6332303, -0.9915643, 0.8291162...","[-0.9370475, -0.48033506, -0.786143, 0.7936281...","[-0.90473247, -0.48250878, -0.5135872, 0.74861...","[-0.89134973, -0.45324185, -0.33538315, 0.7712..."
11577,11577,Journey to Italy (Viaggio in Italia) (Voyage t...,"Drama,Romance",Roberto Rossellini,"Ingrid Bergman, George Sanders","[drama, romance]","[ingrid bergman, george sanders]",[roberto rossellini],drama romance [SEP] roberto rossellini [SEP] i...,drama romance [SEP] roberto rossellini [SEP],drama romance [SEP] ingrid bergman george san...,roberto rossellini [SEP] ingrid bergman georg...,"[-0.9163655, -0.47346175, -0.67750305, 0.83951...","[-0.89984787, -0.44744048, -0.96033573, 0.8897...","[-0.95957094, -0.52262294, -0.8653444, 0.88964...","[-0.89111775, -0.56990063, -0.8846414, 0.75608...","[-0.92171246, -0.5014977, -0.73985416, 0.83781...","[-0.9258048, -0.5171031, -0.7674842, 0.8538953...","[-0.9416288, -0.49623293, -0.8086401, 0.880700..."
28235,28235,Souls at Sea (1937),Adventure,Henry Hathaway,"Gary Cooper,George Raft,Frances Dee,Henry Wilc...",[adventure],"[gary cooper, george raft, frances dee]",[henry hathaway],adventure [SEP] henry hathaway [SEP] gary coop...,adventure [SEP] henry hathaway [SEP],adventure [SEP] gary cooper george raft france...,henry hathaway [SEP] gary cooper george raft f...,"[-0.8732683, -0.37537786, -0.40245366, 0.69441...","[-0.8175434, -0.28633857, 0.32209763, 0.655758...","[-0.84055245, -0.30636835, 0.06770914, 0.70707...","[-0.9024062, -0.66369736, -0.9912699, 0.830529...","[-0.9190777, -0.47337744, -0.7750069, 0.759738...","[-0.85538673, -0.35702837, -0.45868713, 0.6467...","[-0.8098065, -0.25366327, 0.42690387, 0.630348..."
6352,6352,Nuts (1987),Drama,Martin Ritt,"Barbra Streisand, Richard Dreyfuss, Maureen St...",[drama],"[barbra streisand, richard dreyfuss, maureen...",[martin ritt],drama [SEP] martin ritt [SEP] barbra streisand...,drama [SEP] martin ritt [SEP],drama [SEP] barbra streisand richard dreyfuss...,martin ritt [SEP] barbra streisand richard dr...,"[-0.9010864, -0.46036848, -0.51605207, 0.76937...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.86071163, -0.3958799, -0.59048724, 0.73429...","[-0.9061951, -0.5520195, -0.95108724, 0.821535...","[-0.929547, -0.5083384, -0.7278203, 0.81145465...","[-0.9266273, -0.49348006, 

#Test concatenation BERT embeddings

In [72]:
search_phrase_genre="crime drama"

search_tok_genre=tokenizer(search_phrase_genre, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_phrase_director="francis ford coppola"

search_tok_director=tokenizer(search_phrase_director, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre = model(search_tok_genre['input_ids'].to(device), search_tok_genre['attention_mask'].to(device))

search_out_director = model(search_tok_director['input_ids'].to(device), search_tok_director['attention_mask'].to(device))

search_out_genre_director_concat = np.concatenate((search_out_genre[1].detach().cpu().numpy(),search_out_director[1].detach().cpu().numpy()),axis=1)

In [74]:
cosine_sim_search_genre_director_concat = cosine_similarity(np.vstack(df.genre_director_concat_embedding), search_out_genre_director_concat)

In [76]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre_director_concat.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,...,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding,genre_director_concat_embedding,genre_actors_concat_embedding,director_actors_concat_embedding,full_concat_embedding
4425,4425,Tucker: The Man and His Dream (1988),Drama,Francis Ford Coppola,"Jeff Bridges, Frederic Forrest, Martin Landau,...",[drama],"[jeff bridges, frederic forrest, martin landau]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] jeff br...,drama [SEP] francis ford coppola [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.91164315, -0.59014595, -0.94301134, 0.7848...","[-0.8757097, -0.3812641, -0.35433483, 0.724260...","[-0.9148313, -0.41901135, -0.46939766, 0.79699...","[-0.907878, -0.42283192, -0.44207257, 0.798485...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
2006,2006,"Outsiders, The (1983)",Drama,Francis Ford Coppola,"Matt Dillon, C. Thomas Howell, Ralph Macchio, ...",[drama],"[matt dillon, c. thomas howell, ralph macchio]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] matt di...,drama [SEP] francis ford coppola [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9029983, -0.6533675, -0.99136484, 0.851146...","[-0.9394539, -0.6065701, -0.9450279, 0.8594429...","[-0.92072195, -0.60216945, -0.91420037, 0.7974...","[-0.907878, -0.42283192, -0.44207257, 0.798485...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
1591,1591,"Rainmaker, The (1997)",Drama,Francis Ford Coppola,"Matt Damon, Danny DeVito, Jon Voight, Claire D...",[drama],"[matt damon, danny devito, jon voight]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] matt da...,drama [SEP] francis ford coppola [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.93598545, -0.6025171, -0.9919759, 0.854069...","[-0.8867673, -0.37469319, -0.3583872, 0.669029...","[-0.8924352, -0.40954915, -0.39587402, 0.73868...","[-0.907878, -0.42283192, -0.44207257, 0.798485...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
14739,14739,"Rain People, The (1969)",Drama,Francis Ford Coppola,"James Caan, Shirley Knight, Robert Duvall",[drama],"[james caan, shirley knight, robert duvall]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] james c...,drama [SEP] francis ford coppola [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.94587183, -0.6602034, -0.9625455, 0.858021...","[-0.9011832, -0.4551079, -0.45746538, 0.699184...","[-0.88366073, -0.42016616, -0.31581804, 0.7018...","[-0.907878, -0.42283192, -0.44207257, 0.798485...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
8764,8764,Rumble Fish (1983),Drama,Francis Ford Coppola,"Matt Dillon, Mickey Rourke, Diane Lane, Dennis...",[drama],"[matt dillon, mickey rourke, diane lane]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] matt di...,drama [SEP] francis ford coppola [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.93380356, -0.37859926, -0.6878208, 0.81368...","[-0.93551165, -0.6943381, -0.9795263, 0.8

In [77]:
search_phrase_genre="crime drama"

search_tok_genre=tokenizer(search_phrase_genre, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_phrase_actors="marlon brando"

search_tok_actors=tokenizer(search_phrase_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre = model(search_tok_genre['input_ids'].to(device), search_tok_genre['attention_mask'].to(device))

search_out_actors = model(search_tok_actors['input_ids'].to(device), search_tok_actors['attention_mask'].to(device))

search_out_genre_actors_concat = np.concatenate((search_out_genre[1].detach().cpu().numpy(),search_out_actors[1].detach().cpu().numpy()),axis=1)

In [78]:
cosine_sim_search_genre_actors_concat = cosine_similarity(np.vstack(df.genre_actors_concat_embedding), search_out_genre_actors_concat)

In [80]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre_actors_concat.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,...,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding,genre_director_concat_embedding,genre_actors_concat_embedding,director_actors_concat_embedding,full_concat_embedding
14765,14765,"Flight That Fought Back, The (2005)",Drama,Bruce Goodison,Kiefer Sutherland,[drama],[kiefer sutherland],[bruce goodison],drama [SEP] bruce goodison [SEP] kiefer suther...,drama [SEP] bruce goodison [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.65419626, -0.119199924, 0.37860885, 0.3252...","[-0.931274, -0.41297555, -0.6680795, 0.7630407...","[-0.84518695, -0.31740683, -0.29937798, 0.6837...","[-0.92530555, -0.4279519, -0.5756043, 0.824754...","[-0.90155935, -0.40688157, -0.34758177, 0.7842...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.65419626, -0.119199924, 0.37860885, 0.3252...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
39320,39320,Joe Rogan: Live (2006),Comedy,Michael Blieden,Joe Rogan,[comedy],[joe rogan],[michael blieden],comedy [SEP] michael blieden [SEP] joe rogan [...,comedy [SEP] michael blieden [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.8763274, -0.41004395, -0.5580298, 0.654924...","[-0.8923787, -0.44980913, -0.5916888, 0.725004...","[-0.8411285, -0.37258396, -0.39751953, 0.61634...","[-0.90986294, -0.46926695, -0.3993604, 0.76680...","[-0.9003619, -0.46484202, -0.5162596, 0.772190...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.8763274, -0.41004395, -0.5580298, 0.654924...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
39890,39890,Joe Rogan: Rocky Mountain High (2014),Comedy,Ryan Polito,Joe Rogan,[comedy],[joe rogan],[ryan polito],comedy [SEP] ryan polito [SEP] joe rogan [SEP],comedy [SEP] ryan polito [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.8943292, -0.30241567, -0.2815005, 0.712685...","[-0.8923787, -0.44980913, -0.5916888, 0.725004...","[-0.8514758, -0.3213344, -0.16100605, 0.634348...","[-0.90986294, -0.46926695, -0.3993604, 0.76680...","[-0.88758194, -0.39920378, -0.18934739, 0.7643...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.8943292, -0.30241567, -0.2815005, 0.712685...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
48958,48958,Joe Rogan: Strange Times (2018),Comedy,Anthony Giordano,Joe Rogan,[comedy],[joe rogan],[anthony giordano],comedy [SEP] anthony giordano [SEP] joe rogan ...,comedy [SEP] anthony giordano [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.795526, -0.2779253, -0.106529616, 0.594321...","[-0.8923787, -0.44980913, -0.5916888, 0.725004...","[-0.806191, -0.3241982, -0.14326844, 0.5639571...","[-0.90986294, -0.46926695, -0.3993604, 0.76680...","[-0.8767558, -0.41756797, -0.32397994, 0.72668...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.795526, -0.2779253, -0.106529616, 0.594321...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
38382,38382,Joe Rogan: Triggered (2016),Comedy,Anthony Giordano,Joe Rogan,[comedy],[joe rogan],[anthony giordano],comedy [SEP] anthony giordano [SEP] joe rogan ...,comedy [SEP] anthony giordano [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.795526, -0.2779253, -0.106529616, 0.594321...","[-0.8923787, -0.44980913, -0.5916888, 0.725004...","[-0.806191, -0.3241982, -0.14326844, 0.5639571...","[-0.90986294, -0.46926695, -0.3993604, 0.76680...","[-0.8767558, -0.41756797, -0.32397994, 0.72668...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.795526, -0.2779253, -0.106529616, 0.594321...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
18704,18704,"Ring F

In [81]:
search_phrase_director="francis ford coppola"

search_tok_director=tokenizer(search_phrase_director, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_phrase_actors="marlon brando"

search_tok_actors=tokenizer(search_phrase_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_director = model(search_tok_director['input_ids'].to(device), search_tok_director['attention_mask'].to(device))

search_out_actors = model(search_tok_actors['input_ids'].to(device), search_tok_actors['attention_mask'].to(device))

search_out_director_actors_concat = np.concatenate((search_out_director[1].detach().cpu().numpy(),search_out_actors[1].detach().cpu().numpy()),axis=1)

In [82]:
cosine_sim_search_director_actors_concat = cosine_similarity(np.vstack(df.director_actors_concat_embedding), search_out_director_actors_concat)

In [84]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_director_actors_concat.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,...,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding,genre_director_concat_embedding,genre_actors_concat_embedding,director_actors_concat_embedding,full_concat_embedding
41874,41874,Pandora's Box (1992),Drama,Adam Curtis,Adam Curtis,[drama],[adam curtis],[adam curtis],drama [SEP] adam curtis [SEP] adam curtis [SEP],drama [SEP] adam curtis [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.8786718, -0.41367316, -0.22365485, 0.72559...","[-0.86677325, -0.35653362, -0.015761489, 0.734...","[-0.86677325, -0.35653362, -0.015761489, 0.734...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
38561,38561,HyperNormalisation (2016),Documentary,Adam Curtis,Adam Curtis,[documentary],[adam curtis],[adam curtis],documentary [SEP] adam curtis [SEP] adam curti...,documentary [SEP] adam curtis [SEP],...,"[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.8786718, -0.41367316, -0.22365485, 0.72559...","[-0.83925176, -0.2844555, 0.26146787, 0.659989...","[-0.83925176, -0.2844555, 0.26146787, 0.659989...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.8813821, -0.34834626, -0.100846775, 0.7306..."
33305,33305,Location Hunting in Palestine (1965),Documentary,Pier Paolo Pasolini,Pier Paolo Pasolini,[documentary],[pier paolo pasolini],[pier paolo pasolini],documentary [SEP] pier paolo pasolini [SEP] pi...,documentary [SEP] pier paolo pasolini [SEP],...,"[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.9609444, -0.52633655, -0.7517151, 0.878244...","[-0.9609444, -0.52633655, -0.7517151, 0.878244...","[-0.9521272, -0.49434894, -0.7489561, 0.861770...","[-0.9476634, -0.48311293, -0.57739943, 0.86505...","[-0.9476634, -0.48311293, -0.57739943, 0.86505...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.8813821, -0.34834626, -0.100846775, 0.7306...","[-0.9609444, -0.52633655, -0.7517151, 0.878244...","[-0.8813821, -0.34834626, -0.100846775, 0.7306..."
41666,41666,Extraordinary Illusions (1903),Comedy,Georges Méliès,Georges Méliès,[comedy],[georges méliès],[georges méliès],comedy [SEP] georges méliès [SEP] georges méli...,comedy [SEP] georges méliès [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.83744204, -0.33981332, -0.29411465, 0.6869...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
41664,41664,"Bob Kick, the Mischievous Kid (1903)",Comedy,Georges Méliès,Georges Méliès,[comedy],[georges méliès],[georges méliès],comedy [SEP] georges méliès [SEP] georges méli...,comedy [SEP] georges méliès [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.83744204, -0.33981332, -0.29411465, 0.6869...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.9387212, -0.56645

In [86]:
search_phrase_genre="crime drama"

search_tok_genre=tokenizer(search_phrase_genre, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_phrase_director="francis ford coppola"

search_tok_director=tokenizer(search_phrase_director, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_phrase_actors="marlon brando"

search_tok_actors=tokenizer(search_phrase_actors, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

search_out_genre = model(search_tok_genre['input_ids'].to(device), search_tok_genre['attention_mask'].to(device))

search_out_director = model(search_tok_director['input_ids'].to(device), search_tok_director['attention_mask'].to(device))

search_out_actors = model(search_tok_actors['input_ids'].to(device), search_tok_actors['attention_mask'].to(device))

search_out_genre_director_actors_concat = np.concatenate((search_out_genre[1].detach().cpu().numpy(),search_out_director[1].detach().cpu().numpy(),search_out_actors[1].detach().cpu().numpy()),axis=1)

In [88]:
cosine_sim_search_genre_director_actors_concat = cosine_similarity(np.vstack(df.full_concat_embedding), search_out_genre_director_actors_concat)

In [90]:
indices = pd.Series(df['Title'])

def recommend_search(cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim_search_genre_director_actors_concat.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,...,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding,genre_director_concat_embedding,genre_actors_concat_embedding,director_actors_concat_embedding,full_concat_embedding
41874,41874,Pandora's Box (1992),Drama,Adam Curtis,Adam Curtis,[drama],[adam curtis],[adam curtis],drama [SEP] adam curtis [SEP] adam curtis [SEP],drama [SEP] adam curtis [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.8786718, -0.41367316, -0.22365485, 0.72559...","[-0.86677325, -0.35653362, -0.015761489, 0.734...","[-0.86677325, -0.35653362, -0.015761489, 0.734...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9082284, -0.53817147, -0.5488907, 0.764870...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
46527,46527,The Dreyfus Affair (1899),Drama,Georges Méliès,Georges Méliès,[drama],[georges méliès],[georges méliès],drama [SEP] georges méliès [SEP] georges méliè...,drama [SEP] georges méliès [SEP],...,"[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.83744204, -0.33981332, -0.29411465, 0.6869...","[-0.9095947, -0.40867037, -0.49466157, 0.80336...","[-0.9095947, -0.40867037, -0.49466157, 0.80336...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.9428234, -0.61664605, -0.8517865, 0.865877...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.9428234, -0.61664605, -0.8517865, 0.865877..."
50643,50643,"Deduce, You Say (1956)",Animation,Chuck Jones,"Mel Blanc,June Foray",[animation],"[mel blanc, june foray]",[chuck jones],animation [SEP] chuck jones [SEP] mel blanc ju...,animation [SEP] chuck jones [SEP],...,"[-0.9429008, -0.48557085, -0.79645085, 0.84859...","[-0.9193618, -0.50630605, -0.6015173, 0.751280...","[-0.829041, -0.4133765, -0.7776962, 0.6422934,...","[-0.88790524, -0.4635515, -0.5441971, 0.666639...","[-0.9149272, -0.48152813, -0.7043233, 0.793725...","[-0.91150653, -0.3943834, -0.41807818, 0.78178...","[-0.9429008, -0.48557085, -0.79645085, 0.84859...","[-0.9429008, -0.48557085, -0.79645085, 0.84859...","[-0.9193618, -0.50630605, -0.6015173, 0.751280...","[-0.9429008, -0.48557085, -0.79645085, 0.84859..."
41664,41664,"Bob Kick, the Mischievous Kid (1903)",Comedy,Georges Méliès,Georges Méliès,[comedy],[georges méliès],[georges méliès],comedy [SEP] georges méliès [SEP] georges méli...,comedy [SEP] georges méliès [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.83744204, -0.33981332, -0.29411465, 0.6869...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
42234,42234,An Up-to-Date Conjurer (1899),Comedy,Georges Méliès,Georges Méliès,[comedy],[georges méliès],[georges méliès],comedy [SEP] georges méliès [SEP] georges méli...,comedy [SEP] georges méliès [SEP],...,"[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.83744204, -0.33981332, -0.29411465, 0.6869...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.90067995, -0.43202645, -0.56725705, 0.7813...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.89789057, -0.39535615, -0.6202058, 0.79074...","[-0.9387212, -0.56645083, -0.8519647, 0.863428..."
41666,416

#Test simple Bag-of-Words embedding

In [7]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])

In [ ]:
# cosine_sim = cosine_similarity(count_matrix, count_matrix)
# #print(cosine_sim)

# indices = pd.Series(df['Title'])

# def recommend(title, cosine_sim = cosine_sim):
#     recommended_movies = []
#     idx = indices[indices == title].index[0]
#     score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
#     top_10_indices = list(score_series.iloc[1:11].index)
    
#     for i in top_10_indices:
#         recommended_movies.append(list(df['Title'])[i])
    
#     return recommended_movies

# print(recommend('Hare-um Scare-um (1939)'))

In [8]:
count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]","[marlon brando, al pacino, james caan]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] m...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] marlon brando al pacino ja...,francis ford coppola [SEP] marlon brando al p...
1178,1178,"Godfather: Part II, The (1974)","Crime,Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[crime, drama]","[al pacino, robert duvall, diane keaton]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] a...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] al pacino robert duvall di...,francis ford coppola [SEP] al pacino robert d...
1916,1916,"Godfather: Part III, The (1990)","Crime,Drama,Mystery,Thriller",Francis Ford Coppola,"Al Pacino, Diane Keaton, Talia Shire, Andy Gar...","[crime, drama, mystery, thriller]","[al pacino, diane keaton, talia shire]",[francis ford coppola],crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller [SEP] al pacino ...,francis ford coppola [SEP] al pacino diane ke...
7734,7734,Blackboard Jungle (1955),Drama,Richard Brooks,"Glenn Ford, Anne Francis, Sidney Poitier, Vic ...",[drama],"[glenn ford, anne francis, sidney poitier]",[richard brooks],drama [SEP] richard brooks [SEP] glenn ford a...,drama [SEP] richard brooks [SEP],drama [SEP] glenn ford anne francis sidney p...,richard brooks [SEP] glenn ford anne francis ...
4893,4893,"Long Gray Line, The (1955)",Drama,John Ford,"Donald Crisp, Robert Francis, Tyrone Power, Ma...",[drama],"[donald crisp, robert francis, tyrone power]",[john ford],drama [SEP] john ford [SEP] donald crisp robe...,drama [SEP] john ford [SEP],drama [SEP] donald crisp robert francis tyro...,john ford [SEP] donald crisp robert francis ...
19446,19446,Babies for Sale (1940),"Crime,Drama",Charles Barton,"Rochelle Hudson, Glenn Ford, Miles Mander, Isa...","[crime, drama]","[rochelle hudson, glenn ford, miles mander]",[charles barton],crime drama [SEP] charles barton [SEP] rochell...,crime drama [SEP] charles barton [SEP],crime drama [SEP] rochelle hudson glenn ford ...,charles barton [SEP] rochelle hudson glenn fo...
11816,11816,Scum (1979),"Crime,Drama",Alan Clarke,"Ray Winstone, Mick Ford, Julian Firth, John Bl...","[crime, drama]","[ray winstone, mick ford, julian firth]",[alan clarke],crime drama [SEP] alan clarke [SEP] ray winsto...,crime drama [SEP] alan clarke [SEP],crime drama [SEP] ray winstone mick ford jul...,alan clarke [SEP] ray winstone mick ford jul...
2006,2006,"Outsiders, The (1983)",Drama,Francis Ford Coppola,"Matt Dillon, C. Thomas Howell, Ralph Macchio, ...",[drama],"[matt dillon, c. thomas howell, ralph macchio]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] matt di...,drama [SEP] francis ford coppola [SEP],drama [SEP] matt dillon c. thomas howell ral...,francis ford coppola [SEP] matt dillon c. tho...
4425,4425,Tucker: The Man and His Dream (1988),Drama,Francis Ford Coppola,"Jeff Bridges, Frederic Forrest, Martin Landau,...",[drama],"[jeff bridges, frederic forrest, martin landau]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] jeff br...,drama [SEP] francis ford coppola [SEP],drama [SEP] jeff bridges frederic forrest ma...,francis ford coppola [SEP] jeff bridges frede...
8764,8764,Rumble Fish (1983),Drama,Francis Ford Coppola,"Matt Dillon, Mickey Rourke, Diane Lane, Dennis...",[drama],"[matt dillon, mickey rourke, diane lane]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] matt di...,drama [SEP] francis ford coppola [SEP],drama [SEP] matt dillon mickey rourke diane ...,francis ford coppola [SEP] matt dillon mickey...


In [ ]:
df.head()

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors,BoW_embedding,genre_embedding,director_embedding,actors_embedding,director_actors_embedding,genre_actors_embedding,genre_director_embedding
0,0,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...","[adventure, animation, children, comedy, fantasy]","[tim allen, tom hanks, don rickles]",[john lasseter],adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy [S...,john lasseter [SEP] tim allen tom hanks don ...,"[-0.9068614, -0.45405352, -0.80455714, 0.75044...","[-0.8988625, -0.57012534, -0.99096996, 0.91168...","[-0.8687084, -0.30918023, -0.30689827, 0.65527...","[-0.88004625, -0.60375607, -0.98654175, 0.7553...","[-0.8976941, -0.4799076, -0.7705132, 0.7016066...","[-0.9034863, -0.53590953, -0.8715768, 0.737469...","[-0.90445274, -0.47810483, -0.8167057, 0.79308..."
1,1,Jumanji (1995),"Adventure,Children,Fantasy",Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...","[adventure, children, fantasy]","[jonathan hyde, bradley pierce, robin williams]",[joe johnston],adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] joe johnston ...,adventure children fantasy [SEP] jonathan hyde...,joe johnston [SEP] jonathan hyde bradley pier...,"[-0.8920922, -0.3682144, -0.40223628, 0.756093...","[-0.89932764, -0.49810314, -0.98150486, 0.9110...","[-0.7428713, -0.12231341, 0.05766613, 0.381438...","[-0.9077679, -0.58974427, -0.94957626, 0.83750...","[-0.8599445, -0.38748986, -0.42035514, 0.68593...","[-0.916348, -0.40694273, -0.53880054, 0.808105...","[-0.8948911, -0.37579343, -0.5455437, 0.787583..."
2,2,Grumpier Old Men (1995),"Comedy,Romance",Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...","[comedy, romance]","[jack lemmon, walter matthau, ann-margret ]",[howard deutch],comedy romance [SEP] howard deutch [SEP] jack ...,comedy romance [SEP] howard deutch [SEP],comedy romance [SEP] jack lemmon walter matth...,howard deutch [SEP] jack lemmon walter mattha...,"[-0.9042945, -0.46400297, -0.690274, 0.7970322...","[-0.89284724, -0.45970407, -0.9596586, 0.86451...","[-0.82786524, -0.28569412, -0.14196457, 0.6659...","[-0.91705436, -0.5562112, -0.9739642, 0.855578...","[-0.85550654, -0.49105352, -0.8668252, 0.73722...","[-0.92988086, -0.6175473, -0.9531798, 0.882426...","[-0.87577635, -0.38601547, -0.3655597, 0.75878..."
3,3,Waiting to Exhale (1995),"Comedy,Drama,Romance",Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...","[comedy, drama, romance]","[angela bassett, loretta devine, whitney hou...",[forest whitaker],comedy drama romance [SEP] forest whitaker [SE...,comedy drama romance [SEP] forest whitaker [SEP],comedy drama romance [SEP] angela bassett lor...,forest whitaker [SEP] angela bassett loretta ...,"[-0.93603516, -0.5670282, -0.8451405, 0.845204...","[-0.9080508, -0.5593205, -0.98510927, 0.895620...","[-0.9699908, -0.4320408, -0.8304918, 0.9324962...","[-0.90437317, -0.66993964, -0.97878164, 0.8077...","[-0.89529276, -0.5065453, -0.64752454, 0.72272...","[-0.9358029, -0.60015255, -0.87821615, 0.85341...","[-0.8997284, -0.44859746, -0.45308366, 0.81648..."
4,4,Father of the Bride Part II (1995),Comedy,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",[comedy],"[steve martin, martin short, diane keaton]",[charles shyer],comedy [SEP] charles shyer [SEP] steve martin ...,comedy [SEP] charles shyer [SEP],comedy [SEP] steve martin martin short diane...,charles shyer [SEP] steve martin martin short...,"[-0.9054226, -0.5058742, -0.7242107, 0.7743211...","[-0.9387212, -0.56645083, -0.8519647, 0.863428...","[-0.87651086, -0.34650725, -0.5925919, 0.68864...","[-0.8820019, -0.60759795, -0.97453606, 0.80612...","[-0.89944583, -0.52649724, -0.7816844, 0.7549,

#Test Bag-of-Words with Tf-Idf vectorization

In [9]:
TfIdf = TfidfVectorizer()
count_matrix = TfIdf.fit_transform(df['Bag_of_words'])

In [10]:
count_matrix_search=TfIdf.transform(["crime drama [SEP] francis ford coppala"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)

    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Actors_list,Director_List,Bag_of_words,BoW_genre_director,BoW_genre_actors,BoW_director_actors
12674,12674,"Whole Town's Talking, The (Passport to Fame) (...","Comedy,Crime",John Ford,"Edward G. Robinson, Jean Arthur, Wallace Ford,...","[comedy, crime]","[edward g. robinson, jean arthur, wallace ford]",[john ford],comedy crime [SEP] john ford [SEP] edward g. r...,comedy crime [SEP] john ford [SEP],comedy crime [SEP] edward g. robinson jean ar...,john ford [SEP] edward g. robinson jean arthu...
7734,7734,Blackboard Jungle (1955),Drama,Richard Brooks,"Glenn Ford, Anne Francis, Sidney Poitier, Vic ...",[drama],"[glenn ford, anne francis, sidney poitier]",[richard brooks],drama [SEP] richard brooks [SEP] glenn ford a...,drama [SEP] richard brooks [SEP],drama [SEP] glenn ford anne francis sidney p...,richard brooks [SEP] glenn ford anne francis ...
1178,1178,"Godfather: Part II, The (1974)","Crime,Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[crime, drama]","[al pacino, robert duvall, diane keaton]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] a...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] al pacino robert duvall di...,francis ford coppola [SEP] al pacino robert d...
4893,4893,"Long Gray Line, The (1955)",Drama,John Ford,"Donald Crisp, Robert Francis, Tyrone Power, Ma...",[drama],"[donald crisp, robert francis, tyrone power]",[john ford],drama [SEP] john ford [SEP] donald crisp robe...,drama [SEP] john ford [SEP],drama [SEP] donald crisp robert francis tyro...,john ford [SEP] donald crisp robert francis ...
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]","[marlon brando, al pacino, james caan]",[francis ford coppola],crime drama [SEP] francis ford coppola [SEP] m...,crime drama [SEP] francis ford coppola [SEP],crime drama [SEP] marlon brando al pacino ja...,francis ford coppola [SEP] marlon brando al p...
4558,4558,"Cotton Club, The (1984)","Crime,Musical",Francis Ford Coppola,"Richard Gere, Gregory Hines, Diane Lane, Lonet...","[crime, musical]","[richard gere, gregory hines, diane lane]",[francis ford coppola],crime musical [SEP] francis ford coppola [SEP]...,crime musical [SEP] francis ford coppola [SEP],crime musical [SEP] richard gere gregory hine...,francis ford coppola [SEP] richard gere grego...
14739,14739,"Rain People, The (1969)",Drama,Francis Ford Coppola,"James Caan, Shirley Knight, Robert Duvall",[drama],"[james caan, shirley knight, robert duvall]",[francis ford coppola],drama [SEP] francis ford coppola [SEP] james c...,drama [SEP] francis ford coppola [SEP],drama [SEP] james caan shirley knight robert...,francis ford coppola [SEP] james caan shirley...
19101,19101,Fog City Mavericks (2007),Documentary,Gary Leyva,"George Lucas, Francis Ford Coppola",[documentary],"[george lucas, francis ford coppola]",[gary leyva],documentary [SEP] gary leyva [SEP] george luca...,documentary [SEP] gary leyva [SEP],documentary [SEP] george lucas francis ford c...,gary leyva [SEP] george lucas francis ford co...
12142,12142,Youth Without Youth (2007),"Drama,Romance,Sci-Fi",Francis Ford Coppola,"Tim Roth , Alexandra Maria Lara","[drama, romance, sci-fi]","[tim roth , alexandra maria lara]",[francis ford coppola],drama romance sci-fi [SEP] francis ford coppol...,drama romance sci-fi [SEP] francis ford coppol...,drama romance sci-fi [SEP] tim roth alexandr...,francis ford coppola [SEP] tim roth alexandr...
743,743,Jack (1996),"Comedy,Drama",Francis Ford Coppola,"Brian Kerwin, Robin Williams, Diane Lane, Jenn...","[comedy, drama]","[brian kerwin, robin williams, diane lane]",[francis ford coppola],comedy drama [SEP] francis ford coppola [SEP] ...,comedy drama [SEP] francis ford coppola [SEP],comedy drama [SEP] brian kerwin robin william...,francis ford coppola [SEP] brian kerwin robin...
